# Preprossing the dataset

In [1]:
import pandas as pd
import os
from os.path import abspath, join, dirname, normpath
import sys
from skimage import io
import skimage
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
local_path = normpath(abspath(dirname("__file__")))

## covert the breeds into binary

In [2]:
def makelabelset(csv_path):
    landmarks_frame = pd.read_csv(csv_path, sep=',')
    breeds = []
    for name in landmarks_frame.breed:
        if name not in breeds:
            breeds.append(name)
    classes = list(range(len(breeds)))
    labelset = dict(zip(breeds, classes))
    return labelset, breeds

### create user defined dataset 

In [3]:
class DogbreedDataset(Dataset):
    """Load in Dog Breed Dataset"""
    
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied 
                on a sample.
        """
        csv_file = join(local_path, csv_file)
        root_dir = join(local_path, root_dir)
        self.landmarks_frame = pd.read_csv(csv_file, sep=',')
        self.root_dir = root_dir
        self.transform = transform
        self.labelset, _ = makelabelset(csv_file)
        
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        # define the name of pic
        _img_name = self.landmarks_frame.iloc[idx, 0] + '.jpg'
        img_name = os.path.join(self.root_dir, _img_name)
        # read the jpg
#         image = io.imread(img_name)
        image = Image.open(img_name)
#         image = np.array(_image)
        # define the label
        landmarks = self.labelset[self.landmarks_frame.iloc[idx, 1]]
        dataset = {'image': image, 'landmarks': landmarks}
        
        if self.transform:
            dataset = self.transform(dataset)
            
        return dataset

test the defined dataset

In [4]:
dogdataset = DogbreedDataset('all/labels.csv', 'all/train')
dogdataloader = DataLoader(dogdataset, batch_size=4, shuffle=True, num_workers=2)

In [5]:
(dogdataset[100]['landmarks'])

44

### change the size of image 

The images have different height and width.

In [6]:
class Rescale(object):
    """Rescale the image in a sample to a given size.
    
    Args:
        output_size (tuple or int): Desired output size. If tuple, output is 
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        #print (image)
        img = np.asarray(image)
        h,w = img.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
            
        new_h, new_w = int(new_h), int(new_w)
        
        img = image.resize((new_h, new_w))
        #print(img)
        
#         img = Image.fromarray(img)
        
        return {'image': img, 'landmarks': landmarks}

one-hot the dog breeds

In [7]:
class ToTensor(object):
    """Covert ndarrays in sample to Tensors."""
    
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        
#         img = np.asarray(image)
#         img = img.transpose((2, 0, 1))
        trans = transforms.ToTensor()
        img = trans(image)
        #print (type(img))
        # one-hot
        num_classes = 120
        batch_size = 1
        label = torch.LongTensor([[landmarks]])
        landmarks = torch.zeros(batch_size, num_classes).scatter_(1, label, 1)
        #print (type(landmarks))
        return {'image': img, 
                'landmarks': landmarks}

In [8]:
from torchvision.transforms import functional as F
class Normalize(object):
    """Normalize a tensor image with mean and standard deviation.
    Given mean: ``(M1,...,Mn)`` and std: ``(S1,..,Sn)`` for ``n`` channels, this transform
    will normalize each channel of the input ``torch.*Tensor`` i.e.
    ``input[channel] = (input[channel] - mean[channel]) / std[channel]``
    .. note::
        This transform acts in-place, i.e., it mutates the input tensor.
    Args:
        mean (sequence): Sequence of means for each channel.
        std (sequence): Sequence of standard deviations for each channel.
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized Tensor image.
        """
        image, landmarks = sample['image'], sample['landmarks']
        return {'image':F.normalize(image, self.mean, self.std),
                'landmarks': landmarks}

#     def __repr__(self):
#         return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

### Create the dataset

In [9]:
from torchvision import transforms
dogdataset = DogbreedDataset(csv_file='all/labels.csv', 
                             root_dir='all/train', 
                             transform= transforms.Compose([
                                 Rescale((224,224)),
                                 ToTensor(),
                                 Normalize(mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225]),
                             ]))

In [10]:
#dogdataset[100]

### Split the data into train, validation and test

In [11]:
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
def splitdata(dataset, train_ratio, validate_ratio, random_seed):
    dataset_size = len(dataset)
    train_size = int(train_ratio * dataset_size)
    validation_size = int(validate_ratio * dataset_size)
    test_size = int(dataset_size - train_size - validation_size)
    indices = list(range(dataset_size))
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:(train_size + validation_size)]
    test_indices = indices[(train_size + validation_size):]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    return train_sampler, valid_sampler, test_sampler

I leave the random_seed outside for contorling the dataset

In [12]:
train_sampler, valid_sampler, test_sampler = splitdata(dogdataset, 0.7, 0.25, 100)

In [13]:
print (len(train_sampler.indices))

7155


### get the different dataloader

In [14]:
from torch.utils.data import DataLoader
batch_size = 64
train_loader = DataLoader(dogdataset, batch_size=batch_size, sampler=train_sampler, shuffle=False)
validation_loader = DataLoader(dogdataset, batch_size=batch_size, sampler=valid_sampler, shuffle=False)
test_loader = DataLoader(dogdataset, batch_size=batch_size, sampler=test_sampler, shuffle=False)

In [15]:
# dataset_sizes = {'train':len(train_sampler),'valid':len(valid_sampler), 'test':len(test_sampler)}
dataset_sizes = {'train':len(train_sampler),'valid':len(valid_sampler)}

In [16]:
dataloaders = {'train':train_loader,'valid':validation_loader}

# Create a network

In [17]:
from torchvision import models
import torch.nn as nn
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 120)

#if torch.cuda.is_available():
#    model_ft = model_ft.cuda()

In [18]:
#model_ft

### GPU

In [19]:
if torch.cuda.is_available():
    is_cuda = False

### Define the Loss and Optimizer

In [33]:
from torch import optim
from torch.optim import lr_scheduler
import time
learning_rate = 0.001
criterion = nn.MultiLabelSoftMarginLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [44]:
from torch.autograd import Variable
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        losses = []
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data['image'], data['landmarks']

                # wrap them in Variable
                if torch.cuda.is_available():
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                outputs = model(inputs)
                #print (outputs)
                #print (outputs.data)
                _, preds = torch.max(outputs.data, 1)
                
                loss = criterion(outputs, labels)
                print (loss)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                losses.append(loss)
                #losses += losses.append(loss.data.mean())
                
                #running_corrects += torch.sum(preds == labels.data)

            epoch_loss = np.mean(losses)
            #epoch_acc = running_corrects / dataset_sizes[phase]
            epoch_acc = 1 - np.mean(losses)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [45]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=2)

Epoch 0/1
----------


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([64, 1, 120])) that is different to the input size (torch.Size([64, 120])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


tensor(0.7335)
tensor(0.7308)
tensor(0.7278)
tensor(0.7223)
tensor(0.7184)


KeyboardInterrupt: 

https://medium.com/@yvanscher/pytorch-tip-yielding-image-sizes-6a776eb4115b